# Jedi Robes, Inc -- Basic Integer Programming

Suppose Jedi Robes, Inc., top producer of robes for the Jedi Council, makes a set of three items: robes, sandals, and hoods. Each item requires a different machine, which can be rented at the following costs:

* Robes machine costs \\$200/month
* Sandals machine costs \\$150/month
* Hoods machine costs \\$100/month

Jedi Robes, Inc., can choose not to rent a machine, but then they cannot produce any of that item. Robes require 3 hours of labor and 4 square yards of cloth. They sell for \\$12 and cost \$6 to make. Sandals require 6 hours of labor and 4 square yards of cloth. They sell for \\$15 and cost \\$8 to make. Hoods require 2 hours of labor and 3 square yards of cloth. They sell for \\$8 and cost \\$4 to make. There are 150 hours of labor and 160 square yards of cloth available.

How should Jedi Robes, Inc., produce items for the Council?

In [1]:
using JuMP, Gurobi

p = [:robes,:sandals,:hoods]
var_costs = Dict(zip(p,[6 8 4]))
fixed_costs = Dict(zip(p,[200 150 100]))
profit = Dict(zip(p,[12 15 8]))
labor = Dict(zip(p,[3 6 2]))
cloth = Dict(zip(p, [4 4 3]))
labor_avail = 150
cloth_avail = 160

M = 1000 # very large number

m = Model(Gurobi.Optimizer)
set_optimizer_attribute(m,"OutputFlag",0)

@variable(m, x[p] >= 0, Int) # how many do we produce of product p?
@variable(m, z[p], Bin) # do we produce any of product p?

# maximize profit - costs
@objective(m, Max, sum(profit[i]*x[i] - fixed_costs[i]*z[i] - var_costs[i]*x[i] for i in p))

@constraint(m, labor_constr, sum(labor[i]*x[i] for i in p) <= labor_avail) # use only available labor hours
@constraint(m, cloth_constr, sum(cloth[i]*x[i] for i in p) <= cloth_avail) # use only available cloth

@constraint(m, logic_constr[i in p], x[i] <= M*z[i]) # if you don't make product i (z=0), the number you produce must be 0

optimize!(m)

println("Make the following products: " )
for i in p
    println(value(x[i]), " ", i)
end
println("Revenue: ", objective_value(m))

Academic license - for non-commercial use only - expires 2022-06-27
Make the following products: 
0.0 robes
0.0 sandals
53.0 hoods
Revenue: 112.0
